In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'fourier'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_fourier_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,26872.33,-0.15,0.29,0.42,0.02,0.06,0.25,0.46,-0.01,0.09,...,0.01,0.01,-0.01,0.02,0.01,-0.00,0.00,-0.00,0.00,-0.00
3,-0.15,18342.01,0.06,-0.34,0.34,-0.41,0.06,0.12,-0.07,-0.04,...,-0.01,0.02,0.00,0.01,0.00,-0.00,0.00,-0.00,-0.00,-0.00
4,0.29,0.06,13125.04,0.44,0.16,-0.01,0.15,-0.16,0.17,0.00,...,-0.02,0.00,0.01,-0.01,0.00,0.00,0.00,-0.00,0.00,0.00
5,0.42,-0.34,0.44,9342.99,-0.30,0.29,-0.02,-0.08,0.02,0.02,...,-0.01,-0.01,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00
6,0.02,0.34,0.16,-0.30,6062.64,0.01,-0.01,-0.03,-0.08,0.06,...,0.01,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00
7,0.06,-0.41,-0.01,0.29,0.01,3513.53,-0.01,0.01,0.04,-0.02,...,-0.01,0.00,-0.01,0.01,-0.00,0.00,0.00,-0.00,0.00,-0.00
8,0.25,0.06,0.15,-0.02,-0.01,-0.01,2152.14,0.01,-0.06,0.05,...,0.00,0.01,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00
9,0.46,0.12,-0.16,-0.08,-0.03,0.01,0.01,1217.34,-0.03,-0.01,...,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
10,-0.01,-0.07,0.17,0.02,-0.08,0.04,-0.06,-0.03,713.93,0.03,...,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00
11,0.09,-0.04,0.00,0.02,0.06,-0.02,0.05,-0.01,0.03,412.30,...,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1.00000,-0.00001,0.00002,0.00003,0.00000,0.00001,0.00003,0.00008,-0.00000,0.00003,...,0.00001,0.00001,-0.00001,0.00004,0.00002,-0.00000,0.00001,-0.00000,0.00001,-0.00000
3,-0.00001,1.00000,0.00000,-0.00003,0.00003,-0.00005,0.00001,0.00002,-0.00002,-0.00001,...,-0.00001,0.00002,0.00000,0.00002,0.00002,-0.00001,0.00001,-0.00002,-0.00001,-0.00000
4,0.00002,0.00000,1.00000,0.00004,0.00002,-0.00000,0.00003,-0.00004,0.00006,0.00000,...,-0.00003,0.00000,0.00002,-0.00002,0.00000,0.00001,0.00000,-0.00001,0.00001,0.00000
5,0.00003,-0.00003,0.00004,1.00000,-0.00004,0.00005,-0.00000,-0.00002,0.00001,0.00001,...,-0.00001,-0.00002,0.00001,-0.00000,0.00002,0.00000,0.00000,-0.00001,0.00001,0.00001
6,0.00000,0.00003,0.00002,-0.00004,1.00000,0.00000,-0.00000,-0.00001,-0.00004,0.00004,...,0.00001,-0.00000,0.00001,-0.00000,0.00001,0.00002,-0.00001,0.00001,-0.00001,-0.00000
7,0.00001,-0.00005,-0.00000,0.00005,0.00000,1.00000,-0.00001,0.00000,0.00002,-0.00001,...,-0.00003,0.00001,-0.00002,0.00005,-0.00001,0.00002,0.00000,-0.00002,0.00000,-0.00001
8,0.00003,0.00001,0.00003,-0.00000,-0.00000,-0.00001,1.00000,0.00001,-0.00005,0.00005,...,0.00001,0.00003,-0.00001,0.00000,-0.00000,0.00000,-0.00001,-0.00003,-0.00001,-0.00000
9,0.00008,0.00002,-0.00004,-0.00002,-0.00001,0.00000,0.00001,1.00000,-0.00003,-0.00002,...,0.00001,0.00000,-0.00001,0.00003,-0.00001,0.00002,0.00001,0.00001,-0.00001,-0.00001
10,-0.00000,-0.00002,0.00006,0.00001,-0.00004,0.00002,-0.00005,-0.00003,1.00000,0.00005,...,0.00001,-0.00000,0.00000,-0.00001,0.00001,0.00001,-0.00001,0.00001,0.00000,0.00001
11,0.00003,-0.00001,0.00000,0.00001,0.00004,-0.00001,0.00005,-0.00002,0.00005,1.00000,...,0.00001,0.00001,0.00004,-0.00001,0.00003,0.00003,-0.00002,-0.00001,-0.00002,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.7530143952186519

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.14840007e+07 1.46248075e+07 7.46259149e+06 3.76619953e+06
 1.58130616e+06 5.30181767e+05 1.98098637e+05 6.31932066e+04
 2.16364470e+04 7.18570257e+03 2.57924548e+03 9.88414976e+02
 3.60044185e+02 1.20955966e+02 4.04659313e+01 1.12075604e+01
 2.94709759e+00 5.70886574e-01 1.19142276e-01 1.78428445e-02
 3.93329673e-03 6.82344129e-04 1.13652067e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.997811,-0.056366,-0.027582,-0.016865,-0.010115,-0.005648,-0.003414,-0.001909,-0.001129,-0.000648,...,-0.000085,-0.000050,-0.000027,-0.000013,-0.000006,-0.000003,-0.000001,-5.610944e-07,-2.360257e-07,-1.021961e-07
1,0.053508,0.995399,-0.069960,-0.031795,-0.016924,-0.009077,-0.005404,-0.003020,-0.001776,-0.001024,...,-0.000135,-0.000077,-0.000042,-0.000021,-0.000009,-0.000005,-0.000002,-9.367794e-07,-4.478632e-07,-1.521490e-07
2,0.029490,0.065219,0.994085,-0.074319,-0.029118,-0.014183,-0.008249,-0.004595,-0.002658,-0.001541,...,-0.000205,-0.000118,-0.000062,-0.000033,-0.000014,-0.000007,-0.000003,-1.347679e-06,-6.040612e-07,-2.263903e-07
3,0.019825,0.034190,0.069204,0.994478,-0.062183,-0.023478,-0.012991,-0.007073,-0.004089,-0.002353,...,-0.000311,-0.000181,-0.000095,-0.000050,-0.000022,-0.000010,-0.000004,-2.066020e-06,-8.848093e-07,-3.296474e-07
4,0.012630,0.019619,0.030821,0.057582,0.995938,-0.050572,-0.023058,-0.011815,-0.006742,-0.003838,...,-0.000504,-0.000294,-0.000155,-0.000081,-0.000036,-0.000016,-0.000007,-3.180375e-06,-1.622545e-06,-5.709829e-07
5,0.007301,0.010932,0.015780,0.023805,0.046650,0.996284,-0.059099,-0.023082,-0.012342,-0.006933,...,-0.000904,-0.000523,-0.000279,-0.000142,-0.000065,-0.000029,-0.000012,-6.019248e-06,-2.756599e-06,-1.030907e-06
6,0.004731,0.006990,0.009881,0.014263,0.023694,0.055706,0.996143,-0.053567,-0.023527,-0.012436,...,-0.001579,-0.000917,-0.000488,-0.000253,-0.000113,-0.000052,-0.000021,-1.063477e-05,-4.942659e-06,-1.769653e-06
7,0.002778,0.004099,0.005773,0.008166,0.012862,0.024051,0.049776,0.995992,-0.060877,-0.025357,...,-0.002936,-0.001708,-0.000907,-0.000466,-0.000210,-0.000097,-0.000038,-1.888987e-05,-9.134654e-06,-3.387684e-06
8,0.001748,0.002568,0.003575,0.005052,0.007864,0.013888,0.024365,0.056896,0.995339,-0.063181,...,-0.005394,-0.003130,-0.001657,-0.000860,-0.000382,-0.000178,-0.000071,-3.460028e-05,-1.642538e-05,-5.960171e-06
9,0.001066,0.001569,0.002190,0.003077,0.004749,0.008284,0.013792,0.026430,0.058232,0.994308,...,-0.010057,-0.005785,-0.003046,-0.001582,-0.000701,-0.000326,-0.000131,-6.430467e-05,-3.084493e-05,-1.105435e-05


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0021892768028536347,
 0.004600832188770165,
 0.005915225503116628,
 0.005521927783073033,
 0.004062062170340552,
 0.0037157892303457674,
 0.003856934134842338,
 0.004007846243887836,
 0.004660948579873536,
 0.005692327843408562,
 0.0075058714140584515,
 0.008943442041842609,
 0.009562095451599473,
 0.010578358174274882,
 0.011051216648796047,
 0.01125642803853899,
 0.011315638110812598,
 0.01302988174318298,
 0.016058190537292183,
 0.030972830046823163,
 0.06271689782818235,
 0.10655902624217384,
 0.075794832903048]